# This script processes the optim 4 (minRadius - maxNumber) results of the minRadius and maxNumber studies

1. Compile o3memo (includes minRadius=0.1 by default) and o4maxnum (has maxNumber=32 by default) :
```
make bin/o3memo
make bin/o4maxnum
```
2. Run the benchmarking script
```
cd scripts
python3 run_minrad_level.py
python3 run_maxnum_level.py
```
3. The output that we will process is `o4_minradius_<hostname>.csv` and `o4_maxnumber_<hostname>.csv`, that should be already saved in the `Results` folder.



In [ ]:
import os
# get the hostname of the server
hostname = os.popen("hostname").read().strip()
# ensure the directory exists
os.makedirs(hostname, exist_ok=True)
# ensure the files exist
file_list = [                   
                f'o4_minradius_{hostname}.csv',
                f'o4_maxnumber_{hostname}.csv',
            ]

for o4_file in file_list:
    # if it is not in Results
    if not os.path.exists(o4_file):
        # is the file already in the directory?
        assert os.path.exists(os.path.join(hostname, o4_file)), f'File {o4_file} not found: something went wrong with the baseline benchmark.'
    # if it is in Results
    else:
        # copy the file to the directory
        assert os.system(f'mv {o4_file} {hostname}/') == 0, f'Failed to move {o4_file} to {hostname}/'
# add the path to all the names in the list
for i in range(len(file_list)):
    file_list[i] = os.path.join(hostname, file_list[i])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys
from statistics import mean

def tokenize(filename, type='minRadius'):
    results ={}

    with open(filename) as f:
        next(f) # skip header
        for line in f:
            tokens = line.split()
            name=tokens[1].split("/")[3][:-5]
            level=int(tokens[5])
            if type == 'minRadius':
                minRad = float(tokens[3])
            else:
                minRad = int(tokens[4])
            if name not in results:
                results[name]={}
            if level not in results[name]:
                results[name][level]={}
            results[name][level][minRad]=[float(tokens[6])] # tree construction time
            results[name][level][minRad].append(float(tokens[7])) # owm time
            results[name][level][minRad].append(float(tokens[6])+float(tokens[7])) # total time
            results[name][level][minRad].append(float(tokens[8])) # accuracy
    return results, int(tokens[2])

resR,nprocs=tokenize(os.path.join(hostname, f'o4_minradius_{hostname}.csv'), "minRadius")
resN,_=tokenize(os.path.join(hostname, f'o4_maxnumber_{hostname}.csv'), "maxNumber")

# Tree construction execution time for each cloud depending on level and minRadius

In [ ]:
def plot_alltimes(step,results,phase,maxNummode=False):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-','.-','<-','>-','1-','2-','3','4','8','p','h','H','*','D','d','|','_']

    titles=['Tree construction time','OWM time','Total time']
    #fig = plt.figure()
    #labels=[x/10 for x in list(range(1,20,step))]
    if maxNummode:
        labels=list(results["Alcoy"][3].keys())[1:-4:step]
    else:
        labels=list(results["Alcoy"][3].keys())[::step]
    #print(labels)
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in zip(range(4),['Alcoy','Arzua','BrionF','BrionU']):
        x=list(results[name].keys())
        for minrad,m in zip(labels,marks):
            #print(name," ",[results[name][i][minrad][phase] for i in x])
            axs[i].plot(np.array(x), np.array([results[name][i][minrad][phase] for i in x]),m, linewidth=linew, markersize=markers)
        
        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Level', fontsize=xlabelfs)
        axs[i].set_xticks(x[::1])
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()


    fig.suptitle(titles[phase]+' (sec.)',  fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)
    #axs[i].show()
    if(phase==0):
        if(maxNummode):
            plt.savefig(f"{hostname}/MaxNum-Level-{hostname}.pdf")
        else:
            plt.savefig(f"{hostname}/MinRad-Level-{hostname}.pdf")

plot_alltimes(3,resR,0)
plot_alltimes(3,resR,1)
plot_alltimes(3,resR,2)

In [ ]:
#For MaxNumber
plot_alltimes(2,resN,0,True)
plot_alltimes(2,resN,1,True)
plot_alltimes(2,resN,2,True)

# Time for best level [5, 5, 4, 4] for minradius and [6,6,4,4] for maxnumber

In [ ]:
def plot_bestlevel(bestlevel,step,results,figname,maxNummode=False):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-','.-','<-','>-','1-','2-','3','4','8','p','h','H','*','D','d','|','_']

    titles=['OWM Trav.','Tree Const.','Total']
    #fig = plt.figure()
    #labels=[x/10 for x in list(range(1,20,step))]
    if maxNummode:
        x=list(results["Alcoy"][3].keys())[1:-2:step]
    else:
        x=list(results["Alcoy"][3].keys())[::step]
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in zip(range(4),['Alcoy','Arzua','BrionF','BrionU']):
        #x=list(results[name][bestlevel[i]].keys())
        xticks=range(1,len(x)+1)
        for k,m in zip([1,0,2],marks):
            if(maxNummode):
               axs[i].plot(np.array(xticks), np.array([results[name][bestlevel[i]][j][k] for j in x]),m, linewidth=linew, markersize=markers)
            else:
               axs[i].plot(np.array(x), np.array([results[name][bestlevel[i]][j][k] for j in x]),m, linewidth=linew, markersize=markers)
        
        axs[i].set_title(name+" (lev="+str(bestlevel[i])+")",fontsize=16)
        if(maxNummode):
            axs[i].set_xlabel('MaxNumber', fontsize=xlabelfs)
        else:
            axs[i].set_xlabel('MinRadius', fontsize=xlabelfs)
        if(maxNummode):    
            # axs[i].set_xticks(ticks=xticks[::3], labels=x[::3],fontsize=xticksfs)
            axs[i].set_xticks(ticks=xticks[::3], labels=x[::3])
        else:
            axs[i].set_xticks(x[::2])
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()


    fig.suptitle('Time (sec.)',  fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(titles,loc='best', fontsize= 14)
    #axs[i].show()
    pp = PdfPages(figname)
    pp.savefig(fig)
    pp.close()

plot_bestlevel([5,5,4,4], 1, resR, f"{hostname}/MinRad-{nprocs}cores-time-{hostname}.pdf")
plot_bestlevel([6,6,4,4], 1, resN, f"{hostname}/MaxNum-{nprocs}cores-time-{hostname}.pdf",True)

In [ ]:
#For the best level (see next cell) find the min and max accuracy for all minRadius
def findbestlevel_radius(res,phase):
    bestlevel=[]
    for cloud, c in res.items():
        bestlevel.append((cloud,min( ((i, (k0, {k1: i}) ) 
        for k0, d in c.items()
        for k1, i in d.items()), key=lambda x: x[0][phase])[1]))
    for i in range(len(bestlevel)):
        print(bestlevel[i])
    return bestlevel

print("Best level and minRadius for each cloud, minimizing Tree Const. time")
print("Cloud, Level, MinRadius, [Tree Const.,OWM Trav., Total, Accuracy]")
findbestlevel_radius(resR,0)
print("\nBest level and minRadius for each cloud, minimizing OWM traversal time")
print("Cloud, Level, MinRadius, [Tree Const., OWM Trav., Total, Accuracy]")
findbestlevel_radius(resR,1)
print("\nBest level and minRadius for each cloud, minimizing Total time")
print("Cloud, Level, MinRadius, [Tree Const., OWM Trav., Total, Accuracy]")
bestTotalR=findbestlevel_radius(resR,2)

# print ("Cloud & Min accuracy & Max accuracy\\\\ \hline")
# for i,k in zip(res1,[0.7,0.3,0.2,0.1]):
#     accuracy=[res1[i][k][j][3] for j in [1,2,4,6,8]]
#     minaccuracy=min(accuracy)
#     maxaccuracy=max(accuracy)
#     print ("{} & {:0.2f}\% & {:0.2f}\%\\\\ \hline".format(i,minaccuracy,maxaccuracy))

In [ ]:
def findbestlevel_maxnum(res,phase):
    bestlevel=[]
    for cloud, c in res.items():
        bestlevel.append((cloud,min( ((i, (k0, {k1: i}) ) 
        for k0, d in c.items()
        for k1, i in d.items()), key=lambda x: x[0][phase])[1]))
    for i in range(len(bestlevel)):
        print(bestlevel[i])
    return bestlevel

print("Best level and MaxNumber for each cloud, minimizing Tree Const. time")
print("Cloud, Level, MaxNumber, [Tree Const.,OWM Trav., Total, Accuracy]")
findbestlevel_maxnum(resN,0)
print("\nBest level and MaxNumber for each cloud, minimizing OWM traversal time")
print("Cloud, Level, MaxNumber, [Tree Const., OWM Trav., Total, Accuracy]")
findbestlevel_maxnum(resN,1)
print("\nBest level and MaxNumber for each cloud, minimizing Total time")
print("Cloud, Level, MaxNumber, [Tree Const., OWM Trav., Total, Accuracy]")
bestTotalN=findbestlevel_maxnum(resN,2)

# Compute max and min error for different minradius and maxnumber values

In [ ]:
#For the best Level (see previous cell) find the min and max accuracy for all minRadius
print (" & \multicolumn{2}{|c|}{MinRadius} & \multicolumn{2}{|c|}{MaxNumber}\\\\ \hline")
print ("Cloud & Min accuracy & Max accuracy & Min accuracy & Max accuracy\\\\ \hline")
for i,k in zip(resR,[5,5,4,4]):
    accuracyR=[resR[i][k][j][3] for j in resR[i][k].keys()]
    minaccuracyR=min(accuracyR)
    maxaccuracyR=max(accuracyR)
    accuracyN=[resN[i][k][j][3] for j in resN[i][k].keys()]
    minaccuracyN=min(accuracyN)
    maxaccuracyN=max(accuracyN)
    print ("{} & {:0.2f}\% & {:0.2f}\% & {:0.2f}\% & {:0.2f}\%\\\\ \hline".format(i,minaccuracyR,maxaccuracyR,minaccuracyN,maxaccuracyN))

# Add final parallel (n-cores) times to All_Optimizations.out

In [ ]:
print(bestTotalR)
print(bestTotalN)

output = os.path.join(hostname, f'All_Optimizations-{hostname}.csv')

f = open(output, "a")

for i in range(len(bestTotalR)):
    cloud=bestTotalR[i][0] #cloud
    bestl=bestTotalR[i][1][0] #best level   
    bestmr=[x for x in bestTotalR[i][1][1].keys()][0] #best minRadius   
    treet=float(list(bestTotalR[i][1][1].values())[0][0]) #tree time   
    owmt=float(list(bestTotalR[i][1][1].values())[0][1]) #owm time   
    print("Opt4-MinRad;{};{:5f};{:5f};{};{}".format(cloud,treet,owmt,bestl,bestmr))
    f.write("Opt4-MinRad;{};{:5f};{:5f};{};{}\n".format(cloud,treet,owmt,bestl,bestmr))

for i in range(len(bestTotalN)):
    cloud=bestTotalN[i][0] #cloud
    bestl=bestTotalN[i][1][0] #best level   
    bestmn=[x for x in bestTotalN[i][1][1].keys()][0] #best maxNumber   
    treet=float(list(bestTotalN[i][1][1].values())[0][0]) #tree time   
    owmt=float(list(bestTotalN[i][1][1].values())[0][1]) #owm time   
    print("Opt4-MaxNum;{};{:5f};{:5f};{};{}".format(cloud,treet,owmt,bestl,bestmn))
    f.write("Opt4-MaxNum;{};{:5f};{:5f};{};{}\n".format(cloud,treet,owmt,bestl,bestmn))

f.close()